In [47]:
import pandas as pd
import numpy as np
import ast
import datetime
import holidays

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [64]:
usecols = ['CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'TIMESTAMP',
           'dist_perc', 'start_lat', 'start_lon', 'stop_lat', 'stop_lon']

df = pd.read_csv('train_tratado_outliers_2.csv', usecols=usecols, parse_dates=True, nrows=300000)

# cabeçalho minusculo
df.columns = df.columns.map(lambda x: x.lower())
#conversão da data de obj para datetime
df.timestamp = pd.to_datetime(df.timestamp)

In [65]:
def trata_dados(df):
    #--- Timestamp
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    #df['cat_hour'] = pd.cut(df.timestamp.dt.hour, [-1, 8, 16, 23], labels=[0,1,2])
    df['hour'] = df.timestamp.dt.hour
    df['fds_fer'] = df.timestamp.dt.dayofweek
    df['fds_fer'] = df['fds_fer'].apply(lambda x: 1 if x==0 or x==6 else 0) #dom/sab = 0
    for data in sorted(holidays.Portugal(years=[2013, 2014])): # add holidays
        df['fds_fer'] = 1 if len(df[df.timestamp.dt.date==data])>0 else 0
    
    del df['timestamp']

    #--- Call_type hot encode
    df['call_type'] = preprocessing.LabelEncoder().fit_transform(df['call_type'])
    enc = OneHotEncoder().fit_transform(df[['call_type']])
    del df['call_type']
    df['call_type_a'] = enc.toarray()[:,0]
    df['call_type_b'] = enc.toarray()[:,1]
    df['call_type_c'] = enc.toarray()[:,2]

    #--- Fill na
    df.fillna(-1, inplace=True)

    return df

In [66]:
df = trata_dados(df)

In [67]:
df.head()

,origin_call,origin_stand,taxi_id,dist_perc,start_lat,start_lon,stop_lat,stop_lon,month,day,hour,fds_fer,call_type_a,call_type_b,call_type_c
0,-1.0,-1.0,20000589,2.23,41.141412,-8.618643,41.154489,-8.630838,7,1,0,0,0.0,0.0,1.0
1,-1.0,7.0,20000596,3.46,41.159826,-8.639847,41.170671,-8.665740,7,1,0,0,0.0,1.0,0.0
2,-1.0,-1.0,20000320,17.63,41.140359,-8.612964,41.140530,-8.615970,7,1,0,0,0.0,0.0,1.0
3,-1.0,-1.0,20000520,7.97,41.151951,-8.574678,41.142915,-8.607996,7,1,0,0,0.0,0.0,1.0
4,-1.0,-1.0,20000337,4.81,41.180490,-8.645994,41.178087,-8.687268,7,1,0,0,0.0,0.0,1.0


In [72]:
df = df.drop(['origin_call', 'origin_stand'], axis=1)

In [75]:
del df['taxi_id']

## Start ML process

In [5]:
# Deprecated! Using np instead.

#def haversine(lon1, lat1, lon2, lat2):
#    """
#    Calculate the great circle distance between two points 
#    on the earth (specified in decimal degrees)
#    """
#    # convert decimal degrees to radians 
#    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
#
#    # haversine formula 
#    #dlon = lon2 - lon1 
#    #dlat = lat2 - lat1 
#    a = sin((lat2 - lat1)/2)**2 + cos(lat1) * cos(lat2) * sin((lon2 - lon1)/2)**2
#    c = 2 * asin(sqrt(a)) 
#    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
#    return c * r

In [68]:
# Runs some algorithm for comparing

def algs_test(X1, X2, y1, y2):

    algs =[('linear', LinearRegression()),
           ('ridge', Ridge(random_state=99)),
           ('lasso', Lasso(random_state=99)), #lasso 142.7065081365015 561.2795889474457
           ('lsvr', LinearSVR(random_state=99)),
           ('KNN reg', KNeighborsRegressor()), #KNN reg 3.3861972620770264 3.5182287757352873
           ('gboost', GradientBoostingRegressor(random_state=99)),
           ('ada boost', AdaBoostRegressor(random_state=99)),
           ('rnd forest', RandomForestRegressor(random_state=99)),
           ('xgboost', XGBRegressor(random_state=99, n_jobs=-1))
           ]

    for name, alg in algs:
        try:
            alg.fit(X1, y1)
            preds = alg.predict(X2)

            concat = np.apply_along_axis(np.radians, 1, np.concatenate((np.array(y2), preds), axis=1)) 
            hav_dist = np.apply_along_axis(lambda x: 6371*(2*np.arcsin(np.sqrt((np.sin((x[3]-x[1])/2)**2 + np.cos(x[1])*np.cos(x[3])*np.sin((x[2]-x[0])/2)**2)))), 1, concat)
            
            #deprecated!
            #result = y2.copy()
            #result['pred_lat'] = preds[:,0]
            #result['pred_lon'] = preds[:,1]
            #hav_dist = result.apply(lambda x: haversine(x.stop_lon, x.stop_lat, x.pred_lon, x.pred_lat) ,axis=1) 

            print('normal',name, hav_dist.mean(), hav_dist.std())
        except: # se o algoritmo nao possui suporte nativo para multioutput
            mo = MultiOutputRegressor(alg, n_jobs=-1).fit(X1, y1)
            preds = mo.predict(X2)

            concat = np.apply_along_axis(np.radians, 1, np.concatenate((np.array(y2), preds), axis=1)) 
            hav_dist = np.apply_along_axis(lambda x: 6371*(2*np.arcsin(np.sqrt((np.sin((x[3]-x[1])/2)**2 + np.cos(x[1])*np.cos(x[3])*np.sin((x[2]-x[0])/2)**2)))), 1, concat)
            
            #deprecated!
            #result = y2.copy()
            #result['pred_lat'] = preds[:,0]
            #result['pred_lon'] = preds[:,1]
            #hav_dist = result.apply(lambda x: haversine(x.stop_lon, x.stop_lat, x.pred_lon, x.pred_lat) ,axis=1) 

            print('multi',name, hav_dist.mean(), hav_dist.std())

In [69]:
# http://colingorrie.github.io/outlier-detection.html

def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))

def outliers_z_score(ys):
    threshold = 3

    mean_y = np.mean(ys)
    print(mean_y)
    stdev_y = np.std(ys)
    z_scores = [(y - mean_y) / stdev_y for y in ys]
    return np.where(np.abs(z_scores) > threshold)


def outliers_modified_z_score(ys):
    threshold = 3.5

    median_y = np.median(ys)
    median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in ys])
    modified_z_scores = [0.6745 * (y - median_y) / median_absolute_deviation_y
                         for y in ys]
    return np.where(np.abs(modified_z_scores) > threshold)

#------------#
def train_test_mimax(df, remove_outliers=''):
    
    y = df[['stop_lat', 'stop_lon']]
    X = df.drop(['stop_lat', 'stop_lon'], axis=1)
    scaler = MinMaxScaler().fit(X)

    #-- split train/test #X1, X2, y1, y2
    X1, X2, y1, y2 = train_test_split(X, y, test_size=0.3, random_state=42)
    
    if(remove_outliers=='iqr'):
        idx = outliers_iqr(X1.dist_perc)[0].tolist() 
        X1 = X1[~X1.index.isin(idx)]
        y1 = y1[~y1.index.isin(idx)]
    
    if(remove_outliers=='zscore'):
        idx = outliers_z_score(X1.dist_perc)[0].tolist() 
        X1 = X1[~X1.index.isin(idx)]
        y1 = y1[~y1.index.isin(idx)]
    
    if(remove_outliers=='zscore_mod'):
        idx = outliers_modified_z_score(X1.dist_perc)[0].tolist() 
        X1 = X1[~X1.index.isin(idx)]
        y1 = y1[~y1.index.isin(idx)]
        
    #-- fit antes para treinar com todos dados e transform depois para pegar dados de 
    #-- treino com/sem outliers mas treinados com todos dados pra escala ficar igual
    X1 = scaler.transform(X1)
    X2 = scaler.transform(X2)
    
    return X1, X2, y1, y2

**First test (all values)**

In [70]:
algs_test(*train_test_mimax(df))

normal linear 3.7943694900064653 5.16685758992258
normal ridge 4.826768224190662 7.120406616806063
normal lasso 33.39964334808565 273.7373956627983
multi lsvr 3.691853641423378 5.672771679215172
normal KNN reg 4.089013319464276 5.672527741707434
multi gboost 3.2725032218497843 8.499796205125504
multi ada boost 3.8632928094458108 16.2143347486689
normal rnd forest 2.944002064159514 5.366050869011718
multi xgboost 3.2768882403204604 5.0564385057592105


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x0000003BACA4F550>>
Traceback (most recent call last):
  File "C:\Users\a46396\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [76]:
# sem features categoricas
algs_test(*train_test_mimax(df))

normal linear 3.795721884344815 5.169455659748977
normal ridge 4.82580457282064 7.118280071940594
normal lasso 33.39964334808565 273.7373956627983
multi lsvr 3.70924792248832 5.6810188879249415
normal KNN reg 3.568725268940583 5.734704603777714
multi gboost 3.2659645749874127 8.498499044474968
multi ada boost 3.8113126508922885 4.643829697461416
normal rnd forest 2.942860101756264 5.30356548465504
multi xgboost 3.266316939274015 5.051243770495637


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x0000003BB3EB4A90>>
Traceback (most recent call last):
  File "C:\Users\a46396\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


**Removing outliers by distance using IQR**

In [7]:
algs_test(*train_test_mimax(df, 'iqr'))

normal linear 3.7994608185055982 5.169324210494789
normal ridge 4.864851330406091 7.191031409572629
normal lasso 33.33013020703441 273.7456499011859
multi lsvr 3.7104789961152136 5.675317782440048
normal KNN reg 4.091847060703542 5.6517915598768464
multi gboost 3.283240063651977 9.874466442248034
multi ada boost 3.8841356153195066 16.321563749047012
normal rnd forest 2.9511996516451675 7.859806512681066
multi xgboost 3.2797079635768696 5.108205812117166


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x0000003BABBC4550>>
Traceback (most recent call last):
  File "C:\Users\a46396\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


**Removing outliers by distance using Z-Score**

In [8]:
algs_test(*train_test_mimax(df, 'zscore'))

5.413344857142967
normal linear 3.7959447007924907 5.165570018422575
normal ridge 4.829782203103702 7.125578541880782
normal lasso 33.44289094100121 273.73226416601483
multi lsvr 3.693475209870266 5.679699736418575
normal KNN reg 4.089335190659967 5.672224862701648
multi gboost 3.2727887580506776 7.3455606129823945
multi ada boost 3.7837968592551503 16.231941395528896
normal rnd forest 2.9396363804846013 5.402115547034383
multi xgboost 3.2753886421055585 5.027189211676691


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x0000003BABC85F28>>
Traceback (most recent call last):
  File "C:\Users\a46396\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


**Removing outliers by distance using Z-Score modified**

In [9]:
algs_test(*train_test_mimax(df, 'zscore_mod'))

normal linear 3.7983740043591676 5.1693522981615745
normal ridge 4.85907227165783 7.181538294760816
normal lasso 33.40905143203647 273.7362827482903
multi lsvr 3.7020345828925296 5.699824635823981
normal KNN reg 4.091837725827818 5.652503759898603
multi gboost 3.2689681088448346 7.742069152314645
multi ada boost 3.8220556526982246 4.697962267777119
normal rnd forest 2.9578621237774656 9.099826792432346
multi xgboost 3.2685970231398436 5.087903724375411


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x0000003BAB2A75F8>>
Traceback (most recent call last):
  File "C:\Users\a46396\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


Remover os outliers não fez grandes mudanças no modelo. Os melhores foram os ensemble. Vamos usar o Gradient boost e Random Forest como vencedores.


In [8]:
def haversine_scorer(y_real, y_pred):
    concat = np.apply_along_axis(np.radians, 1, np.concatenate((np.array(y_real), y_pred), axis=1)) 
    return np.apply_along_axis(lambda x: 6371*(2*np.arcsin(np.sqrt((np.sin((x[3]-x[1])/2)**2 + np.cos(x[1])*np.cos(x[3])*np.sin((x[2]-x[0])/2)**2)))), 1, concat).mean()

In [9]:
score_ = make_scorer(haversine_scorer, greater_is_better=False)

In [14]:
param_grid = {'max_depth':[10, 40],
              'min_samples_leaf':[50, 150]}

gs = GridSearchCV(RandomForestRegressor(n_estimators=100, random_state=99, n_jobs=-1), param_grid=param_grid, n_jobs=-1, cv=10, scoring=score_, verbose=2)

In [15]:
y = df[['stop_lat', 'stop_lon']]
X = df.drop(['stop_lat', 'stop_lon'], axis=1)
X = MinMaxScaler().fit_transform(X)

In [ ]:
gs.fit(X,y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


#####################

In [12]:
y = df[['stop_lat', 'stop_lon']]
X = df.drop(['stop_lat', 'stop_lon'], axis=1)
X = MinMaxScaler().fit_transform(X)

X1, X2, y1, y2 = train_test_split(X, y, test_size=0.3, random_state=42)

reg = RandomForestRegressor(n_jobs=-1, random_state=99, verbose=2)
reg.fit(X1, y1)

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   16.9s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=99, verbose=2, warm_start=False)

In [13]:
score_(reg, X2, y2)

[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.7s finished


-2.9426354585071417